#Setting

In [ ]:
!pip install transformers==4.17 datasets accelerate evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 35.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 35.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 38.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 36.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 21.0 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 22.3 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.2
    Uninstalling transformers-4.35.2:
      Successfully uninstalled transformers-4.35.2


In [ ]:
from datasets import load_dataset

dataset = load_dataset('klue', 'dp')
dataset

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/10000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['sentence', 'index', 'word_form', 'lemma', 'pos', 'head', 'deprel'],
        num_rows: 10000
    })
    validation: Dataset({
        features: ['sentence', 'index', 'word_form', 'lemma', 'pos', 'head', 'deprel'],
        num_rows: 2000
    })
})

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Fine-tune a pretrained model

## Prepare a dataset

In [ ]:
dataset["train"][0]

{'sentence': '해당 그림을 보면 디즈니 공주들이 브리트니 스피어스의 앨범이나 뮤직비디오, 화보 속 모습을 똑같이 재연했다.',
 'index': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14],
 'word_form': ['해당',
  '그림을',
  '보면',
  '디즈니',
  '공주들이',
  '브리트니',
  '스피어스의',
  '앨범이나',
  '뮤직비디오,',
  '화보',
  '속',
  '모습을',
  '똑같이',
  '재연했다.'],
 'lemma': ['해당',
  '그림 을',
  '보 면',
  '디즈니',
  '공주 들 이',
  '브리트니',
  '스피어스 의',
  '앨범 이나',
  '뮤직 비디오 ,',
  '화보',
  '속',
  '모습 을',
  '똑같이',
  '재연 하 였 다 .'],
 'pos': ['NNG',
  'NNG+JKO',
  'VV+EC',
  'NNP',
  'NNG+XSN+JKS',
  'NNP',
  'NNP+JKG',
  'NNG+JC',
  'NNG+NNG+SP',
  'NNG',
  'NNG',
  'NNG+JKO',
  'MAG',
  'NNG+XSA+EP+EF+SF'],
 'head': [2, 3, 14, 5, 14, 7, 10, 10, 10, 11, 12, 14, 14, 0],
 'deprel': ['NP',
  'NP_OBJ',
  'VP',
  'NP',
  'NP_SBJ',
  'NP',
  'NP_MOD',
  'NP_CNJ',
  'NP_CNJ',
  'NP',
  'NP',
  'NP_OBJ',
  'AP',
  'VP']}

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("klue/bert-base")

Downloading:   0%|          | 0.00/289 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/425 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/243k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [ ]:
tokenizer.is_fast

True

In [ ]:
dataset["train"][0]["word_form"]

['해당',
 '그림을',
 '보면',
 '디즈니',
 '공주들이',
 '브리트니',
 '스피어스의',
 '앨범이나',
 '뮤직비디오,',
 '화보',
 '속',
 '모습을',
 '똑같이',
 '재연했다.']

In [ ]:
#이미 tokenize 된 경우 is_split_into_words = True 로 설정
inputs = tokenizer(dataset["train"][0]["sentence"], is_split_into_words=False)
print(inputs.tokens())

['[CLS]', '해당', '그림', '##을', '보', '##면', '디즈니', '공주', '##들이', '브리', '##트니', '스피', '##어스', '##의', '앨범', '##이나', '뮤직', '##비', '##디오', ',', '화보', '속', '모습', '##을', '똑같이', '재연', '##했', '##다', '.', '[SEP]']
